In [38]:
import requests
import pandas as pd
import os


In [35]:
region = {10 : "Eastern England",
11 : "East Midlands",
12 : "London",
13 : "North Wales & Mersey",
14 : "Midlands",
15 : "North East",
16 : "North West",
17 : "Northern Scotland",
18 : "Southern Scotland",
19 : "South East",
20 : "Southern",
21 : "South Wales",
22 : "South Western",
23 : "Yorkshire"}

In [48]:
def fetch_data(dno_region_code, start_date, end_date):
    
    headers = {
        "Accept": "application/json"
        }

    r = requests.get(f"https://odegdcpnma.execute-api.eu-west-2.amazonaws.com/development/prices?dno={dno_region_code}&voltage=HV&start={start_date}&end={end_date}",
                    params={}, 
                    headers = headers)

    response = r.json()

    df = pd.json_normalize(response['data']['data'])

    df = df.rename(columns={'Timestamp':'timestamp','Overall':'price'})

    df = df.drop(columns=['unixTimestamp'],axis=1)

    df['region_code'] = dno_region_code

    df['region_name'] = region[dno_region_code]

    df = df[['region_code','region_name','timestamp','price']]

    os.mkdir(f'Users/gagandeepkundi/Education/data-engineering/data-engineering-projects/energy_data_uk/data_files/{region[dno_region_code]}')

    return df.to_csv(f'/Users/gagandeepkundi/Education/data-engineering/data-engineering-projects/energy_data_uk/data_files/{region[dno_region_code]}/{start_date}-{region[dno_region_code]}.csv',index=False)

In [49]:
for region in range(10, 24):
    for start_date in pd.date_range(start='1/1/2020', periods=12, freq='M'):
        for end_date in pd.date_range(end='1/1/2020', periods=12, freq='M'):
            fetch_data(region, start_date=f"{start_date}", end_date=f"{end_date}")

KeyError: 'data'